In [1]:
import json as js
from tqdm import tqdm
import string
import sys
import keras
import tensorflow as tf
import json
import numpy as np
import dask.array as da
import math
import os
import dask.array as da
os.environ["CUDA_DEVICE_ORDER"]="0"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from rnn_bimodel import RNNBimodel
from text_processing import textProcessing
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer as Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

Using TensorFlow backend.


In [2]:

if "all_descriptions_train.zarr" in os.listdir('.') and "embedding_train_128.zarr" in os.listdir('.'):
        embedding_train = da.from_zarr("embedding_train_128.zarr")
        desc_train = da.from_zarr("all_descriptions_train.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()
print('dONE')
if "all_descriptions_validation.zarr" in os.listdir('.') and "embedding_validation_128.zarr" in os.listdir('.'):
        embedding_validation = da.from_zarr("embedding_validation_128.zarr")
        desc_validation = da.from_zarr("all_descriptions_validation.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()
print('dONE')
if "all_descriptions_test.zarr" in os.listdir('.') and "embedding_test_128.zarr" in os.listdir('.'):
        embedding_test = da.from_zarr("embedding_test_128.zarr")
        desc_test = da.from_zarr("all_descriptions_test.zarr")
else: 
    print("Embedding and Descriptions dask array haven't been saved, please run text_preprocessing.py")
    exit()

dONE
dONE


In [3]:
desc_train

dask.array<from-zarr, shape=(1000, 5), dtype=<U212, chunksize=(827, 5), chunktype=numpy.ndarray>

In [4]:
# allText = np.append(desc_train.compute(),desc_validation.compute())
# allText = np.append(allText,desc_test.compute())
allText = []
for desc_array in [desc_train.compute(), desc_train.compute(), desc_validation.compute()]:
    for descs in desc_array:
        for desc in descs:
            allText.append(desc)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(allText)
vocab_size = len(tokenizer.word_index) + 1

In [5]:
max_length = max(len(d.split()) for d in allText)
# [word for word in allText if len(word.split())>30]
max_length

43

In [6]:
def count_length(tokenizer, descriptions):
    Y = 0
    for i in tqdm(range(len(descriptions))):
        seq = tokenizer.texts_to_sequences([(descriptions[i].compute()).tolist()])[0]
        Y+=len(seq)-1
    return Y

In [7]:
if 'best_model_rnn.h5' in os.listdir():
    model = load_model('best_model_rnn.h5')
else:
    model = RNNBimodal.build(vocab_size, max_length)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 256)      1426688     input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 80)           0           input_1[0][0]                    
______________________________________________________________________________________________

In [8]:
epochs = 50
nb_train_samples = count_length(tokenizer, desc_train)
nb_validation_samples = count_length(tokenizer, desc_validation)
print(nb_train_samples, nb_validation_samples)

100%|███████████████████████████████████████████████████████████████████████████| 25013/25013 [00:37<00:00, 671.03it/s]


489691 245186


In [16]:
with tf.device('/cpu:0'):
    stop_early = EarlyStopping(monitor='val_loss',patience = 10)
    ModelCheck = ModelCheckpoint('best_model_rnn.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    callbacks = [stop_early, ModelCheck]
    for i in range(epochs):
        train_generator = textProcessing.data_generator(desc_train, embedding_train, tokenizer, max_length, vocab_size)
        validation_generator = textProcessing.data_generator(desc_validation, embedding_validation, tokenizer, max_length, vocab_size)
        model.fit_generator(
            train_generator, 
            epochs=1, 
            steps_per_epoch=nb_train_samples//32, 
            verbose=1)
    

   22/15302 [..............................] - ETA: 2:05:40 - loss: 9.39 - ETA: 1:37:52 - loss: 9.40 - ETA: 1:29:52 - loss: 9.41 - ETA: 1:24:34 - loss: 9.41 - ETA: 1:21:14 - loss: 9.40 - ETA: 1:19:17 - loss: 9.40 - ETA: 1:17:06 - loss: 9.40 - ETA: 1:16:06 - loss: 9.39 - ETA: 1:15:25 - loss: 9.39 - ETA: 1:14:48 - loss: 9.39 - ETA: 1:14:44 - loss: 9.39 - ETA: 1:14:24 - loss: 9.39 - ETA: 1:13:51 - loss: 9.38 - ETA: 1:13:28 - loss: 9.37 - ETA: 1:13:16 - loss: 9.37 - ETA: 1:12:58 - loss: 9.36 - ETA: 1:12:50 - loss: 9.35 - ETA: 1:12:35 - loss: 9.34 - ETA: 1:12:30 - loss: 9.32 - ETA: 1:12:27 - loss: 9.31 - ETA: 1:12:20 - loss: 9.28 - ETA: 1:13:25 - loss: 9.2604

SystemError: <built-in function TFE_Py_TapeGradient> returned a result with an error set

In [8]:
# from text_processing import textProcessing
# train_generator = textProcessing.data_generator(desc_train, embedding_train, tokenizer, max_length, vocab_size)
# a,b = next(train_generator)
# print(type(next(train_generator)))
# print(a[0].shape)
# print(a[1].shape)
# print(b.shape)

In [19]:
model  = load_model('best_model_rnn.h5')

text = textProcessing.generate_desc(model, tokenizer, np.array([embedding_test[0].compute()]), max_length)

print(desc_test[0].compute(),'\n\n', text)

['startseq white plate topped with sandwich and chips endseq'
 'startseq sandwich is sitting on white plate on table endseq'
 'startseq plate of sandwich cut into parts with pasta in middle of it endseq'
 'startseq picture filled with many things all inside endseq'
 'startseq club sandwich sits on plate with bacon endseq'] 

 startseq two white and white and white and and white and and white endseq
